In [1]:
!pip install scrapy
!pip install praw
!pip install pyodbc
!pip install coinmarketcap_scraper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.4/286.4 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 67.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.1/247.1 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.1/97.1 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.0/191.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.8/334.8 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 MB 11.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for coinmarketcap_scraper: filename=coinmarketcap_scraper-1.0.0-py3-none-any.whl size=2971 sha256=369c80a157eb6c4e339a529bfee52a3a9e7e63560b68e9f36d099153a1d9f205
  Stored in directory: /root/.cache/pip/wheels/c7/58/1e/d447d3b92d6ced294d6870fc00110a486

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import scrapy
import praw
import pandas as pd
import nltk
import pprint
import datetime as dt
import json
import pyodbc
import tweepy
import re
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import spacy
import requests

In [4]:
from datetime import datetime
from datetime import date
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk import re
from nltk.probability import FreqDist
from coinmarketcap_scraper import *
from bs4 import BeautifulSoup
from urllib.request import urlopen
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
from nltk.stem.snowball import SnowballStemmer
from curses.ascii import isdigit
from spacy import displacy
from nltk.corpus import brown
from bs4 import BeautifulSoup

It currently works on windows only
It currently works on windows only


Clase para almacenar títulos de noticias y sitios asociados

In [5]:
class Bd_news:
    def __init__(self, t, b, u, s, n, d):
        self.title = t
        self.body = b
        self.url = u
        self.site = s
        self.ner = n
        self.date = d

    def __str__(self):
        return f"{self.title} {self.body}  {self.ner} {self.url} {self.site} {self.date}"

    def to_dict(self):
        return {
            'title': self.title,
            'site': self.site,
            'body': self.body,
            'url': self.url,
            'date': self.date,
            'ner': self.ner
        }

Clase para almacenar palabras, entidades nombradas y frecuencia asociada

In [6]:
class Bd_words:
    def __init__(self, w, f, t, s, d):
        self.word = w
        self.frequency = f
        self.tags = t
        self.id_sentimiento = s
        self.date = d

    def __str__(self):
        return f"{self.word} {self.frequency} {self.date}"

In [7]:
results_noticias = []
results_palabras = []

Empiezo a scrapear Reddit, Bing

In [8]:
#IDENTIFICACION REDDIT
user_agent = "NLP 1.0 by /u/ImaginationNational"
reddit = praw.Reddit(
    client_id="l2be_IkknycgJj8-w2RcOw",
    client_secret="gfKOqiXXVvVSAx1eyXSS886XpDlD2w",
    user_agent=user_agent,
)

Itero a través de las publicaciones en la subreddit 'bitcoin'. Para cada publicación, almaceno el título y la marca de tiempo (timestamp) en la lista results_scrap. También creo objetos de la clase Bd_news para cada publicación y los almacenas en la lista result_for_db.

In [9]:
def reddit_search(query):
  news_data = []
  for submission in reddit.subreddit(query).new(limit=10):
    if submission.selftext and submission.selftext.strip():
      news_data.append({"title": submission.title, "body": submission.selftext, "url": submission.url, "site": 1})
  return news_data

In [10]:
results_noticias_reddit = reddit_search("bitcoin")

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Defino esta funcion de Bing, donde el parametro es la palabra a buscar en los titulos de las noticias

In [11]:
def bing_search(query):
  headers = {
      "User-Agent":
      "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.19582"
    }
  market = "en-US"
  regex = re.compile(r'^https://')
  html = requests.get(f"https://www.bing.com/news/search?q={query}&mkt={market}", headers=headers)
  soup = BeautifulSoup(html.text, 'lxml')

  news_data=[]
  o = {}
  for result in soup.select('.card-with-cluster'):
      article_url = result.select_one('a')['href']
      if regex.match(article_url):
        article_response = requests.get(article_url, headers=headers)
      else:
        break
      article_soup = BeautifulSoup(article_response.content, "html.parser")
      article_paragraphs = article_soup.find_all("p")
      article_title = article_soup.title.text.strip()
      article_body = " ".join(paragraph.text for paragraph in article_paragraphs)
      news_data.append({"title": article_title, "body": article_body, "url": article_url, "site": 2})
  return news_data

In [12]:
query = "bitcoin"
results_noticias_bing = bing_search(query)
results_noticias_bing

[{'title': 'MSN',
  'body': '',
  'url': 'https://www.msn.com/en-us/money/other/bitcoin-may-see-a-supply-shock-as-available-tokens-hit-their-lowest-level-since-2018-with-a-potential-etf-likely-to-jolt-demand-analyst-says/ar-AA1iXLMt',
  'site': 2},
 {'title': 'thestreet.com',
  'body': 'Please enable JS and disable any ad blocker',
  'url': 'https://www.thestreet.com/crypto/innovation/new-book-raises-questions-about-britney-spears-use-of-bitcoin-to-hide-purchases',
  'site': 2},
 {'title': 'thestreet.com',
  'body': 'Please enable JS and disable any ad blocker',
  'url': 'https://www.thestreet.com/crypto/markets/public-miners-weigh-in-why-is-a-potential-etf-propping-the-bitcoin-price',
  'site': 2},
 {'title': 'MSN',
  'body': '',
  'url': 'https://www.msn.com/en-us/money/markets/if-you-invested-1000-in-bitcoin-and-other-crypto-when-jim-cramer-said-anyone-holding-was-an-idiot-heres-how-much-youd-have-now/ar-AA1iXFSl',
  'site': 2},
 {'title': 'Bitcoin mining stocks are poised for a fal

Defino esta funcion de Google

In [13]:
def google_search(query):
  headers = {
      "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
  }
  from googlesearch import search
  help(search)
  results_urls_google = search(query,stop=2,tpe="nws")
  news_data = []
  for item in results_urls_google:
      article_url = item
      article_response = requests.get(article_url, headers=headers)
      article_soup = BeautifulSoup(article_response.content, "html.parser")
      article_paragraphs = article_soup.find_all("p")
      article_title = article_soup.title.text.strip()
      article_body = " ".join(paragraph.text for paragraph in article_paragraphs)
      news_data.append({"title": article_title, "body": article_body, "url": article_url, "site": 3})
  return news_data

In [14]:
results_noticias_google = google_search('bitcoin')
results_noticias_google

Help on function search in module googlesearch:

search(query, tld='com', lang='en', tbs='0', safe='off', num=10, start=0, stop=None, domains=None, pause=2.0, tpe='', country='', extra_params=None, user_agent=None)
    Search the given query string using Google.
    
    :param str query: Query string. Must NOT be url-encoded.
    :param str tld: Top level domain.
    :param str lang: Language.
    :param str tbs: Time limits (i.e "qdr:h" => last hour,
        "qdr:d" => last 24 hours, "qdr:m" => last month).
    :param str safe: Safe search.
    :param int num: Number of results per page.
    :param int start: First result to retrieve.
    :param int stop: Last result to retrieve.
        Use None to keep searching forever.
    :param list domains: A list of web domains to constrain
        the search.
    :param float pause: Lapse to wait between HTTP requests.
        A lapse too long will make the search slow, but a lapse too short may
        cause Google to block your IP. Your mi

[{'title': 'Bitcoin Price (BTC) Gained Nearly 20% This Week, Catching Market Cap of Tesla (TSLA)',
  'body': "TV & Videos Watch On Twitter Facebook Sign up for our newsletters The SBF Trial Daily recaps from crypto’s biggest trial. First Mover The latest moves in crypto markets, in context. The Node The biggest crypto news and ideas of the day. State of Crypto Probing the intersection of crypto and government. The Protocol Exploring the tech behind crypto. Crypto Long & Short News and analysis for the professional investor. Crypto for Advisors What financial advisors need to know about crypto. Money Reimagined The transformation of value in the digital age. By signing up, you will receive emails about CoinDesk products and you agree to our terms & conditions and privacy policy. Podcasts Events Indices Markets Finance Policy Technology Learn Crash Courses Consensus Magazine Most Recent Issue Protocol Village Sponsored Content Bitcoin market cap catches up to some big names (Noah Sillima

Junto todas las noticias de las 3 APIs

In [15]:
listas_noticias = [results_noticias_reddit, results_noticias_bing, results_noticias_google]
results_palabras = []
results_noticias = []
for lista_noticias in listas_noticias:
    for noticia in lista_noticias:
        aux_bd = Bd_news(noticia['title'], noticia['body'], noticia['url'], noticia['site'], [], date.today())
        texto_completo = noticia['title'] + ' ' + noticia['body']
        results_palabras.append(texto_completo)
        results_noticias.append(aux_bd)
results_noticias

Creo un DF con todas las noticias y les asigno un valor de sentimiento

In [16]:
df_database_noticias = pd.DataFrame([s.__dict__ for s in results_noticias])

Empiezo a analizar los sentimientos de las palabras con SIA (Sentiment Intensity Analyzer). Calculo la polaridad del sentimiento, sea negativo,positivo o neutral, almacenadolos para cada noticia.

In [17]:
nltk.download('vader_lexicon')
sia = SIA ()
results = []

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


Empiezo a guardar el sentimiento asociado para la base de datos, comparando porcentajes y determinando el sentimiento dominante

id 1 - positivo , 2 - neutral, 3 - negativo

In [18]:
def obtener_sentimiento(texto):
  sentiment_scores = sia.polarity_scores(texto)
  if sentiment_scores['compound'] >= 0.05:
      return 1
  elif sentiment_scores['compound'] <= -0.05:
      return 3
  else:
      return 2

Cargo el modelo preentrenado de spaCy para el análisis del lenguaje en inglés que puede reconocer nombres, organizaciones.... Luego itero para cada noticia, donde aplicO el modelo NER, como por ultimo agregandolo a una lista nueva. Para visualizar las entidades en las noticias utilizo displacy.render(), style="ent" para visualizar y jupyter=True para un entorno tipo jupyter

In [19]:
NER = spacy.load("en_core_web_sm")


In [20]:
def obtener_ner(texto):
  text_ner= NER(texto)
  array_con_ner = []
  for word in text_ner.ents:
    array_con_ner.append({'tag': word.label_, 'word': word.text})
  return array_con_ner

# CALCULO SENTIMIENTO DE LA NOTICIA EN GENERAL POR EL BODY

In [21]:
df_database_noticias['id_sentimiento'] = df_database_noticias['body'].apply(obtener_sentimiento)
df_database_noticias['ner'] = df_database_noticias['body'].apply(obtener_ner)
df_database_noticias

,title,body,url,site,ner,date,id_sentimiento
0,Do companies that do crypto processing and pay...,Its a payment processor for crypto that pays o...,https://www.reddit.com/r/Bitcoin/comments/17ht...,1,[],2023-10-27,2
1,How secure are seed phrases?,https://x.com/coletu/status/171793689356262639...,https://www.reddit.com/r/Bitcoin/comments/17ht...,1,[],2023-10-27,2
2,Canadian Exchanges still able to trade usdt?,I used to use kraken but now they sent out a m...,https://www.reddit.com/r/Bitcoin/comments/17hs...,1,"[{'tag': 'ORG', 'word': 'USDT'}, {'tag': 'CARD...",2023-10-27,1
3,Forced Bitcoin exposure,MSTR now has over 150k BTC in its coffers. Thi...,https://www.reddit.com/r/Bitcoin/comments/17hr...,1,"[{'tag': 'ORG', 'word': 'MSTR'}, {'tag': 'ORG'...",2023-10-27,3
4,The US is f*cked. If you thought civil asset f...,Its disgusting that cops can take your cash ju...,https://i.redd.it/vjahqtvqjrwb1.jpg,1,"[{'tag': 'CARDINAL', 'word': 'two'}, {'tag': '...",2023-10-27,3
5,New to Bitcoin? Read Moneyhappyhour.com,Informative website with basically a summary o...,http://Moneyhappyhour.com,1,"[{'tag': 'TIME', 'word': 'the couple thousand ...",2023-10-27,1
6,MSN,,https://www.msn.com/en-us/money/other/bitcoin-...,2,[],2023-10-27,2
7,thestreet.com,Please enable JS and disable any ad blocker,https://www.thestreet.com/crypto/innovation/ne...,2,[],2023-10-27,1
8,thestreet.com,Please enable JS and disable any ad blocker,https://www.thestreet.com/crypto/markets/publi...,2,[],2023-10-27,1
9,MSN,,https://www.msn.com/en-us/money/markets/if-you...,2,[],2023-10-27,2


# CALCULO EL SENTIMIENTO DEL DIA A APARTIR DE LAS PALABRAS DE LAS NOTICIAS

Creo un diccionario vacio, recorro la lista utilizando la funcion enumerate para poder utilizar el indice y el valor del elemento

In [22]:
dic = {}
for i,e in enumerate(results_palabras):
  dic[i] = [e]
print(dic)

{0: ['Do companies that do crypto processing and payout in crypto without holding onto the crypto more than to just verify the transactions need to follow FinCEN laws around KYC and AML? Its a payment processor for crypto that pays out directly without them holding it but Im having a issue with them and theyre claiming non-liable.'], 1: ['How secure are seed phrases? https://x.com/coletu/status/1717936893562626394?s=46&t=JOkx2ceOFf4FzkwEEZqraA'], 2: ['Canadian Exchanges still able to trade usdt? I used to use kraken but now they sent out a message they are discontinuing USDT trades\n\n&#x200B;\n\nanyone in canada please recommend a good exchange to trade usdt with?\n\n&#x200B;\n\nthanks!'], 3: ['Forced Bitcoin exposure MSTR now has over 150k BTC in its coffers. This astronomical amount of BTC should be concerning to their competitors.\n\nThat’s because if there will be another bull run, competitors of MSTF will be at risk of becoming a hostile takeover target. There is however one grea

Creo un dataframe a partir del diccionario

In [23]:
dataframe_palabras = pd.DataFrame.from_dict(dic)
dataframe_palabras

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,Do companies that do crypto processing and pay...,How secure are seed phrases? https://x.com/col...,Canadian Exchanges still able to trade usdt? I...,Forced Bitcoin exposure MSTR now has over 150k...,The US is f*cked. If you thought civil asset f...,New to Bitcoin? Read Moneyhappyhour.com Inform...,MSN,thestreet.com Please enable JS and disable any...,thestreet.com Please enable JS and disable any...,MSN,Bitcoin mining stocks are poised for a fall if...,MSN,Leak Reveals Joe Biden Could Be About To Issue...,MSN,Bitcoin Price (BTC) Gained Nearly 20% This Wee...,They Cracked the Code to a Locked USB Drive Wo...


Convierto todo a string (texto) para luego dividirlo en palabras

In [24]:
df_string = dataframe_palabras.to_string()
df_split = df_string.split()
df_split

['0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '0',
 'Do',
 'companies',
 'that',
 'do',
 'crypto',
 'processing',
 'and',
 'payout',
 'in',
 'crypto',
 'without',
 'holding',
 'onto',
 'the',
 'crypto',
 'more',
 'than',
 'to',
 'just',
 'verify',
 'the',
 'transactions',
 'need',
 'to',
 'follow',
 'FinCEN',
 'laws',
 'around',
 'KYC',
 'and',
 'AML?',
 'Its',
 'a',
 'payment',
 'processor',
 'for',
 'crypto',
 'that',
 'pays',
 'out',
 'directly',
 'without',
 'them',
 'holding',
 'it',
 'but',
 'Im',
 'having',
 'a',
 'issue',
 'with',
 'them',
 'and',
 'theyre',
 'claiming',
 'non-liable.',
 'How',
 'secure',
 'are',
 'seed',
 'phrases?',
 'https://x.com/coletu/status/1717936893562626394?s=46&t=JOkx2ceOFf4FzkwEEZqraA',
 'Canadian',
 'Exchanges',
 'still',
 'able',
 'to',
 'trade',
 'usdt?',
 'I',
 'used',
 'to',
 'use',
 'kraken',
 'but',
 'now',
 'they',
 'sent',
 'out',
 'a',
 'message',
 'they',
 'are',
 'discontinuing',
 

Filtro el texto utilizando expresiones regulares para mantener solo palabras y símbolos relevantes excluyendo @user.

In [25]:
lines = [re.sub(r'[^($|#)A-Za-z0-9]', '', x) for x in df_split]
lines2 = []
for word in lines:
    if word != '' and word.lower() != 'rt':
        lines2.append(word)
print(lines2)

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '0', 'Do', 'companies', 'that', 'do', 'crypto', 'processing', 'and', 'payout', 'in', 'crypto', 'without', 'holding', 'onto', 'the', 'crypto', 'more', 'than', 'to', 'just', 'verify', 'the', 'transactions', 'need', 'to', 'follow', 'FinCEN', 'laws', 'around', 'KYC', 'and', 'AML', 'Its', 'a', 'payment', 'processor', 'for', 'crypto', 'that', 'pays', 'out', 'directly', 'without', 'them', 'holding', 'it', 'but', 'Im', 'having', 'a', 'issue', 'with', 'them', 'and', 'theyre', 'claiming', 'nonliable', 'How', 'secure', 'are', 'seed', 'phrases', 'httpsxcomcoletustatus1717936893562626394s46tJOkx2ceOFf4FzkwEEZqraA', 'Canadian', 'Exchanges', 'still', 'able', 'to', 'trade', 'usdt', 'I', 'used', 'to', 'use', 'kraken', 'but', 'now', 'they', 'sent', 'out', 'a', 'message', 'they', 'are', 'discontinuing', 'USDT', 'tradesnn#x200Bnnanyone', 'in', 'canada', 'please', 'recommend', 'a', 'good', 'exchange', 'to', 'trade', 'usd

Elimino las palabras comunes del texto, digitos

In [26]:
nltk.download('stopwords')
array_to_analyze = []
stop_word = set(stopwords.words('english'))
for word in lines2:
    if word not in stop_word:
        if not word[0].isdigit(): #Elimino todos los dígitos sueltos
            array_to_analyze.append(word)
array_to_analyze

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


['Do',
 'companies',
 'crypto',
 'processing',
 'payout',
 'crypto',
 'without',
 'holding',
 'onto',
 'crypto',
 'verify',
 'transactions',
 'need',
 'follow',
 'FinCEN',
 'laws',
 'around',
 'KYC',
 'AML',
 'Its',
 'payment',
 'processor',
 'crypto',
 'pays',
 'directly',
 'without',
 'holding',
 'Im',
 'issue',
 'theyre',
 'claiming',
 'nonliable',
 'How',
 'secure',
 'seed',
 'phrases',
 'httpsxcomcoletustatus1717936893562626394s46tJOkx2ceOFf4FzkwEEZqraA',
 'Canadian',
 'Exchanges',
 'still',
 'able',
 'trade',
 'usdt',
 'I',
 'used',
 'use',
 'kraken',
 'sent',
 'message',
 'discontinuing',
 'USDT',
 'tradesnn#x200Bnnanyone',
 'canada',
 'please',
 'recommend',
 'good',
 'exchange',
 'trade',
 'usdt',
 'withnn#x200Bnnthanks',
 'Forced',
 'Bitcoin',
 'exposure',
 'MSTR',
 'BTC',
 'coffers',
 'This',
 'astronomical',
 'amount',
 'BTC',
 'concerning',
 'competitorsnnThats',
 'another',
 'bull',
 'run',
 'competitors',
 'MSTF',
 'risk',
 'becoming',
 'hostile',
 'takeover',
 'target',

Empiezo a analizar los sentimientos de las palabras con SIA (Sentiment Intensity Analyzer) de cada palabra. Calculo la polaridad del sentimiento, sea negativo,positivo o neutral, almacenadolos para cada palabra.

In [27]:
sia = SIA ()
array_palabras = []

for line in array_to_analyze:
    pol_score = sia.polarity_scores(line)
    pol_score['headline'] = line
    array_palabras.append(pol_score)

In [28]:
array_palabras

[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0, 'headline': 'Do'},
 {'neg': 0.0,
  'neu': 1.0,
  'pos': 0.0,
  'compound': 0.0,
  'headline': 'companies'},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0, 'headline': 'crypto'},
 {'neg': 0.0,
  'neu': 1.0,
  'pos': 0.0,
  'compound': 0.0,
  'headline': 'processing'},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0, 'headline': 'payout'},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0, 'headline': 'crypto'},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0, 'headline': 'without'},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0, 'headline': 'holding'},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0, 'headline': 'onto'},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0, 'headline': 'crypto'},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0, 'headline': 'verify'},
 {'neg': 0.0,
  'neu': 1.0,
  'pos': 0.0,
  'compound': 0.0,
  'headline': 'transactions'},
 {'neg': 0.0, 'neu': 1.0, 'pos'

Creo un dataframe de los resultados del analisis Compound = puntuacion compuesta

In [29]:
dataframe_palabras = pd.DataFrame.from_records(array_palabras)
dataframe_palabras

,neg,neu,pos,compound,headline
0,0.0,1.0,0.0,0.0,Do
1,0.0,1.0,0.0,0.0,companies
2,0.0,1.0,0.0,0.0,crypto
3,0.0,1.0,0.0,0.0,processing
4,0.0,1.0,0.0,0.0,payout
...,...,...,...,...,...
3342,0.0,1.0,0.0,0.0,permission
3343,0.0,1.0,0.0,0.0,Cond
3344,0.0,1.0,0.0,0.0,Nast
3345,0.0,1.0,0.0,0.0,Ad


In [30]:
dataframe_palabras['label'] = 0
dataframe_palabras.loc[dataframe_palabras['compound'] > 0.2 , 'label'] = 1
dataframe_palabras.loc[dataframe_palabras['compound'] < -0.2, 'label'] = -1
dataframe_palabras

,neg,neu,pos,compound,headline,label
0,0.0,1.0,0.0,0.0,Do,0
1,0.0,1.0,0.0,0.0,companies,0
2,0.0,1.0,0.0,0.0,crypto,0
3,0.0,1.0,0.0,0.0,processing,0
4,0.0,1.0,0.0,0.0,payout,0
...,...,...,...,...,...,...
3342,0.0,1.0,0.0,0.0,permission,0
3343,0.0,1.0,0.0,0.0,Cond,0
3344,0.0,1.0,0.0,0.0,Nast,0
3345,0.0,1.0,0.0,0.0,Ad,0


Calculo el porcentaje de cada tipo de sentimientos.

In [31]:
sentiment_prom = dataframe_palabras.label.value_counts(normalize=True) * 100
sentiment_prom

 0    90.498954
 1     7.170601
-1     2.330445
Name: label, dtype: float64

In [ ]:
#sentiment_id = 2
#if(sentiment_prom.count() == 3):
#    if((sentiment_prom[0] > sentiment_prom[1]) and (sentiment_prom[0] > sentiment_prom[-1])): #Si el sentimiento neutro es mayor que los otros dos
#        sentiment_id = 2
#    elif ((sentiment_prom[1] > sentiment_prom[0]) and (sentiment_prom[1] > sentiment_prom[-1])): #Cuandos sea positivo
#        sentiment_id = 1
#    else:
#        sentiment_id = 3
#else:
#     if((sentiment_prom[0] > sentiment_prom[1])):
#        sentiment_id = 2
#     elif ((sentiment_prom[1] > sentiment_prom[0])):
#        sentiment_id = 1
#     else:
#        sentiment_id = 3

In [32]:
sentiment_id= 1
if(sentiment_prom[1] > sentiment_prom[-1]):
  sentiment_id = 1
else:
  sentiment_id = 3


AGARAR SOLO LAS PALABRAS CON SENTIMIENTOS

In [33]:
indices_a_mantener = [idx for idx, palabra in enumerate(array_palabras) if palabra['compound'] != 0.0]
array_filtrado = [array_to_analyze[idx] for idx in indices_a_mantener]
print(array_filtrado)

['AML', 'secure', 'please', 'recommend', 'good', 'Forced', 'risk', 'hostile', 'great', 'well', 'limited', 'threat', 'well', 'defence', 'create', 'defenses', 'play', 'fails', 'asset', 'bad', 'disgusting', 'winnings', 'incentives', 'fucked', 'free', 'hard', 'legal', 'forced', 'allow', 'prison', 'fucked', 'friends', 'like', 'Please', 'Please', 'poised', 'Credit', 'Credit', 'Help', 'Low', 'Credit', 'Credit', 'Credit', 'Best', 'Credit', 'Best', 'Rewards', 'Credit', 'Best', 'Credit', 'Best', 'Credit', 'Best', 'Credit', 'Best', 'Credit', 'Best', 'Credit', 'Welcome', 'Bonuses', 'Best', 'Credit', 'Credit', 'Best', 'Best', 'Best', 'Debt', 'Best', 'Credit', 'Debt', 'Best', 'Best', 'Best', 'Best', 'Best', 'Best', 'Best', 'Best', 'Best', 'No', 'No', 'Best', 'Bonuses', 'Best', 'Best', 'Best', 'Credit', 'Best', 'Best', 'Best', 'No', 'Best', 'No', 'Best', 'Credit', 'Best', 'Best', 'Best', 'Best', 'Credit', 'Best', 'Credit', 'Best', 'Boost', 'Credit', 'Credit', 'Best', 'Best', 'Best', 'Best', 'Pay', 'B

Busco la frecuencia de cada palabra, luego itero sobre cada noticia, aumentando el contador si su palabra ya se encuentra en el objeto

In [34]:
freqdoctor = FreqDist()

for words in array_filtrado:
    freqdoctor[words] += 1

freqdoctor

FreqDist({'Best': 59, 'Credit': 34, 'worth': 8, 'secure': 5, 'free': 5, 'like': 5, 'certain': 5, 'Please': 4, 'Debt': 4, 'No': 4, ...})

Obtengo las 20 palabras más frecuentes

In [35]:
word_more_freq = []
word_more_freq = freqdoctor.most_common(20)
word_more_freq

[('Best', 59),
 ('Credit', 34),
 ('worth', 8),
 ('secure', 5),
 ('free', 5),
 ('like', 5),
 ('certain', 5),
 ('Please', 4),
 ('Debt', 4),
 ('No', 4),
 ('Bad', 4),
 ('intelligence', 4),
 ('help', 4),
 ('lost', 4),
 ('risk', 3),
 ('well', 3),
 ('want', 3),
 ('ready', 3),
 ('successfully', 3),
 ('reached', 3)]

In [36]:
def encontrar_id_sentimiento(sentimiento):
        if sentimiento['neu'] == 1.0:
            return 2
        elif sentimiento['pos'] == 1.0:
            return 1
        elif sentimiento['neg'] == 1.0:
            return 3

In [37]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
words = []
for w in word_more_freq:
    text = word_tokenize(w[0])
    pos_tags = nltk.pos_tag(text)
    tags = [(word, tag) for word, tag in pos_tags]
    id_sentiment = encontrar_id_sentimiento(sia.polarity_scores(w[0]))
    bd_word_obj = Bd_words(w[0], w[1], json.dumps(tags), id_sentiment, date.today())
    words.append(bd_word_obj)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


Creo un DataFrame a partir de los objetos Bd_words para luego guardarlo en la base de datos

In [38]:
df_database_palabras = pd.DataFrame([s.__dict__ for s in words])
df_database_palabras

,word,frequency,tags,id_sentimiento,date
0,Best,59,"[[""Best"", ""JJS""]]",1,2023-10-27
1,Credit,34,"[[""Credit"", ""NN""]]",1,2023-10-27
2,worth,8,"[[""worth"", ""NN""]]",1,2023-10-27
3,secure,5,"[[""secure"", ""NN""]]",1,2023-10-27
4,free,5,"[[""free"", ""JJ""]]",1,2023-10-27
5,like,5,"[[""like"", ""IN""]]",1,2023-10-27
6,certain,5,"[[""certain"", ""JJ""]]",1,2023-10-27
7,Please,4,"[[""Please"", ""VB""]]",1,2023-10-27
8,Debt,4,"[[""Debt"", ""NN""]]",3,2023-10-27
9,No,4,"[[""No"", ""DT""]]",3,2023-10-27


# CONEXION CON API COINMARKETCAP

Hago una solicitud a la API de CoinMarketCap para obtener información sobre el precio de Bitcoin.

In [39]:
headers = {
        'X-CMC_PRO_API_KEY': 'ee1d8880-b02f-4424-872d-14e625332de3',
        'Accepts': 'application/json'
}

In [40]:
params = {
        'start': '1',
        'limit': '6',
        'convert': 'USD'
}

In [41]:
url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
json = requests.get(url, params = params, headers = headers).json()

coins =  json['data']

In [42]:
name = coins[0]['name']
price = coins[0]['quote']['USD']['price']

# Guardo toda la información en la Base de Datos

In [46]:
import sqlite3

In [47]:
conn = sqlite3.connect("/content/drive/MyDrive/PLN/pln.db")

In [48]:
cursor = conn.cursor()


In [ ]:
cursor.execute('''
    CREATE TABLE Price (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT,
        price TEXT,
        date_btc DATETIME
    )
''')
cursor.execute('''
    CREATE TABLE Sentimiento (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        sentimiento TEXT
    )
''')

cursor.execute('''
    CREATE TABLE Bitcoin_Sentimiento (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        time_stamp DATETIME,
        id_sentimiento INTEGER
    )
''')



In [ ]:
sentimientos_lista = ['positivo', 'neutral', 'negativo']
for sentimiento in sentimientos_lista:
    cursor.execute('''
        INSERT INTO Sentimiento (sentimiento) VALUES (?)
    ''', (sentimiento,))

In [ ]:
cursor.execute('''
    CREATE TABLE Sitios_informacion (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        nombre TEXT
    )
''')

sitios = ['reddit', 'bing', 'google']
for sitio in sitios:
    cursor.execute('''
        INSERT INTO Sitios_informacion (nombre) VALUES (?)
    ''', (sitio,))

In [49]:
def create_price(conn):
    cursor = conn.cursor()
    cursor.execute(
        'SELECT * FROM Price WHERE date_btc = ?',
        (datetime.today().strftime('%Y-%m-%d'),)
    )
    existing_row = cursor.fetchone()

    if existing_row is None:
        # La fila no existe, realizar la inserción
        cursor.execute(
            'INSERT INTO Price (name, price, date_btc) VALUES (?,?,?)',
            (name, price, datetime.today().strftime('%Y-%m-%d'))
        )
        conn.commit()
        print("Fila insertada correctamente.")
    else:
        # La fila ya existe, no hacer nada o manejar según tus necesidades
        print("La fila ya existe para la fecha de hoy.")

In [50]:
def create_information(conn):
    cursor = conn.cursor()
    cursor.execute(
        'INSERT INTO Bitcoin_Sentimiento (time_stamp, id_sentimiento) VALUES (strftime("%Y-%m-%d %H:%M:%S", "now", "localtime"), ?)',
        (sentiment_id,)
    )
    conn.commit()

Seleccioné los precios de la criptomoneda entre las fechas especificadas (fecha_inicio y fecha_fin) y calculo la variación de precio. Luego, compara esta variación con el sentimiento predecido e imprime un mensaje correspondiente.

In [51]:
def read_price(conn):
    cursor = conn.cursor()
    cursor.execute(
        'SELECT name, price, strftime("%Y-%m-%d", date_btc) FROM Price'
    )
    for row in cursor:
        print(row)
    print()
    conn.commit()

In [52]:
def select_all_notices():
  cursor = conn.cursor()
  cursor.execute('''
  SELECT Noticias.title, Noticias.body, Noticias.date, Sitios_informacion.nombre AS nombre_sitio, Sentimiento.sentimiento AS sentimiento, Noticias.ner
    FROM Noticias
    JOIN Sitios_informacion ON Noticias.site = Sitios_informacion.id
    JOIN Sentimiento ON Noticias.id_sentimiento = Sentimiento.id''')
  resultados = cursor.fetchall()
  data = []
  for row in resultados:
      title, body, date, nombre_sitio, sentimiento, ner = row
      data.append({"title": title, "body": body, "date": date, "nombre_sitio": nombre_sitio, "sentimiento": sentimiento, "ner": ner})
  df = pd.DataFrame(data)
  return df

In [53]:
def select_notices():
  cursor = conn.cursor()
  cursor.execute('''
  SELECT * from Noticias''')
  resultados = cursor.fetchall()
  for row in resultados:
    print(row)

In [54]:
def take_bitcoin_sent(fecha):
    cursor = conn.cursor()
    cursor.execute(
        'SELECT id_sentimiento FROM Bitcoin_Sentimiento WHERE DATE(time_stamp) = ?',
        (fecha,)
    )
    sentimiento = cursor.fetchone()

    return sentimiento[0]


In [55]:
def analize_price(fecha_inicio, fecha_fin):
    cursor = conn.cursor()
    cursor.execute('SELECT CAST(price AS DECIMAL) FROM Price WHERE date_btc BETWEEN ? AND ?', (fecha_inicio, fecha_fin))
    resultados = cursor.fetchall()
    if len(resultados) >= 2:
        price_anterior = resultados[0][0]
        price_actual = resultados[-1][0]
        variacion_precio = (price_actual - price_anterior)
        sentimiento = take_bitcoin_sent(fecha_inicio)
        print(sentimiento, variacion_precio )
        # Comparar el sentimiento predecido con la variación de precio
        if((variacion_precio > 0) and (sentimiento == 1)):
          print('El sentimiento predecido coincide con un aumento en el precio.')
        elif((variacion_precio < 0)  and (sentimiento == 3)):
          print('El sentimiento predecido coincide con una disminución en el precio.')
        elif((variacion_precio > 0) and (sentimiento == 3)):
          print('El sentimiento predecido no coincide con un aumento en el precio.')
        elif((variacion_precio < 0)  and (sentimiento == 1)):
          print('El sentimiento predecido no coincide con una disminución en el precio.')
        else:
          print('El sentimiento predecido coincide con una variación neutra en el precio.')
    else:
        print('No hay suficientes datos para analizar la variación de precio.')

In [56]:
def select_all_words():
  cursor = conn.cursor()
  cursor.execute('''SELECT Palabras.word, Palabras.frequency, Palabras.tags, Palabras.date, Sentimiento.sentimiento AS sentimiento
  FROM Palabras
  JOIN Sentimiento ON Palabras.id_sentimiento = Sentimiento.id''')
  resultados = cursor.fetchall()
  data = []
  for row in resultados:
      word, frequency, tags, date, sentimiento = row
      data.append({"word": word, "frequency": frequency, "tags": tags, "date": date, "sentimiento": sentimiento})
  df = pd.DataFrame(data)
  return df

In [57]:
create_price(conn)

La fila ya existe para la fecha de hoy.


In [58]:
read_price(conn)

('Bitcoin', '30108.9218793515', '2023-10-21')
('Bitcoin', '30665.1155046492', '2023-10-23')
('Bitcoin', '34032.5690863428', '2023-10-24')
('Bitcoin', '34722.5511702196', '2023-10-26')
('Bitcoin', '34102.8414006232', '2023-10-27')



In [ ]:
create_information(conn)

In [ ]:
fecha_inicio = '2023-10-26'
fecha_fin = '2023-10-27'
analize_price(fecha_inicio, fecha_fin)

1 -619.7097695964039
El sentimiento predecido no coincide con una disminución en el precio.


In [59]:
import json
df_database_noticias['ner'] = df_database_noticias['ner'] = df_database_noticias['ner'].apply(json.dumps)

In [60]:
df_database_noticias.to_sql('Noticias', conn, if_exists='append', index=False)

16

INSERTO LAS NOTICIAS Y LAS PALABRAS ANALIZADAS EN LA BASE DE DATOS

In [63]:
df_database_palabras.to_sql('Palabras', conn, if_exists='append', index=False)

20

In [64]:
df = select_all_notices()
df

,title,body,date,nombre_sitio,sentimiento,ner
0,Interview: Rodolfo Andragnes on Labitconf 2023...,Argentinian elections are a big talking point ...,2023-10-26,reddit,positivo,"[{""tag"": ""NORP"", ""word"": ""Argentinian""}, {""tag..."
1,So suggesting that someone gets off of 0% byc ...,"Wow, I would guess it was because I had the wo...",2023-10-26,reddit,positivo,[]
2,Bitcoin is the ultimate scarcity!,"Like many people here, I felt slightly more di...",2023-10-26,reddit,positivo,"[{""tag"": ""ORG"", ""word"": ""DCA""}, {""tag"": ""ORDIN..."
3,FinCen,I can't help but think that the FinCen is beh...,2023-10-26,reddit,positivo,"[{""tag"": ""PRODUCT"", ""word"": ""FinCen""}, {""tag"":..."
4,The halving is less than 6 months away. Act ac...,The halving is < 6 months away.\n\nBitcoin has...,2023-10-26,reddit,negativo,"[{""tag"": ""DATE"", ""word"": ""6 months""}, {""tag"": ..."
5,100% Non-institutional,"I've taken the final steps, this month and las...",2023-10-26,reddit,positivo,"[{""tag"": ""DATE"", ""word"": ""this month""}, {""tag""..."
6,Want to switch from coinbase (US based)? Strik...,With the negative press coinbase has gotten in...,2023-10-26,reddit,positivo,"[{""tag"": ""DATE"", ""word"": ""the past year""}]"
7,Ledger to Colcard,I have my Bitcoin on a Ledger and have not upg...,2023-10-26,reddit,positivo,"[{""tag"": ""PERSON"", ""word"": ""Bitcoin""}, {""tag"":..."
8,MSN,,2023-10-26,bing,neutral,[]
9,Leak Reveals Joe Biden Could Be About To Issue...,Bitcoin\nBTC\n and major cryptocurrenices incl...,2023-10-26,bing,negativo,"[{""tag"": ""ORG"", ""word"": ""XRP""}, {""tag"": ""DATE""..."


In [65]:
df = select_all_words()
df

,word,frequency,tags,date,sentimiento
0,Best,59,"[[""Best"", ""JJS""]]",2023-10-27,positivo
1,Credit,34,"[[""Credit"", ""NN""]]",2023-10-27,positivo
2,worth,8,"[[""worth"", ""NN""]]",2023-10-27,positivo
3,secure,5,"[[""secure"", ""NN""]]",2023-10-27,positivo
4,free,5,"[[""free"", ""JJ""]]",2023-10-27,positivo
5,like,5,"[[""like"", ""IN""]]",2023-10-27,positivo
6,certain,5,"[[""certain"", ""JJ""]]",2023-10-27,positivo
7,Please,4,"[[""Please"", ""VB""]]",2023-10-27,positivo
8,Debt,4,"[[""Debt"", ""NN""]]",2023-10-27,negativo
9,No,4,"[[""No"", ""DT""]]",2023-10-27,negativo


In [62]:
  cursor = conn.cursor()
  cursor.execute('DROP TABLE IF EXISTS palabras')

In [ ]:
  cursor.execute('DROP TABLE IF EXISTS noticias')


In [66]:
conn.close()